# KAIROS-ARK: The Operating System for Agentic AI

Welcome to the official demo notebook for **KAIROS-ARK**.

This notebook demonstrates the core capabilities of the KAIROS kernel:
1.  **Deterministic Execution** (Hello World)
2.  **True Parallelism** (Multi-threaded implementation)
3.  **Security Policy Engine** (Kernel-level restrictions)

> **Note:** This runs entirely natively in Rust. No Python logic is executed in the scheduler hot path.

## 1. Installation

First, we install the package from PyPI.

In [ ]:
!pip install kairos-ark

## 2. Hello World Agent

Let's create a simple deterministic agent. KAIROS-ARK guarantees that running this agent with the same seed will produce **bit-for-bit identical** audit logs, every single time.

In [ ]:
from kairos_ark import Agent
import json

# 1. Initialize Agent with a fixed seed for determinism
agent = Agent(seed=42)

# 2. Define simple tools (nodes)
agent.add_node("fetch_data", lambda: {"data": "raw_input_from_sensor"})
agent.add_node("process_data", lambda input: {"status": "processed", "value": input["data"].upper()})

# 3. Connect the workflow graph
agent.connect("fetch_data", "process_data")

# 4. Execute starting from 'fetch_data'
print("🚀 Executing Workflow...")
results = agent.execute("fetch_data")

print("\n✅ Execution Complete!")
print("Results:", json.dumps(results, indent=2))

## 3. True Parallel Execution

Unlike standard Python `asyncio` loops which are single-threaded (GIL-bound), KAIROS-ARK uses a **Rust Rayon Thread Pool**. This allows true parallel fan-out for CPU-bound or blocking I/O tasks.

Here, we simulate 3 parallel tasks running simultaneously.

In [ ]:
import time

# Define a blocking task that sleeps
def heavy_task(name):
    print(f"[{name}] Starting...")
    time.sleep(1.0) # Sleep for 1 second
    print(f"[{name}] Done!")
    return f"{name}_result"

# Add parallel nodes
agent.add_node("task_a", lambda: heavy_task("A"))
agent.add_node("task_b", lambda: heavy_task("B"))
agent.add_node("task_c", lambda: heavy_task("C"))

# Create a Fork-Join structure
# Fork: Splits execution flow to multiple nodes
agent.add_fork("start_parallel", ["task_a", "task_b", "task_c"])

# Join: Waits for all branches to complete before proceeding
agent.add_join("sync_results", ["task_a", "task_b", "task_c"])

print("\n⚡ Starting Parallel Execution (Expect ~1.0s total time, NOT 3.0s)...")
start_time = time.time()

agent.execute("start_parallel")

end_time = time.time()
print(f"\n⏱️ Total Time: {end_time - start_time:.4f} seconds")

## 4. Kernel-Level Security Policy

One of KAIROS-ARK's unique features is its **Policy Engine**. You can restrict what an agent can do (network access, file system) at the kernel level, making it safer than just "trusting the prompt".

In [ ]:
from kairos_ark import Policy, Cap

# 1. Define a Restrictive Policy
# - Allow: Only LLM calls
# - Forbid: Network Access, File System
# - Redact: Any content containing "API_KEY"
policy = Policy(
    allowed_capabilities=[Cap.LLM_CALL],
    forbidden_content=["API_KEY"],
    max_tool_calls={"sensitive_tool": 0} # Block specific tool
)

# 2. Run Agent with Policy applied
print("\n🛡️  Running with Security Policy...")
try:
    # Attempting an operation that might violate policy (simulated)
    # In a real connector, checking 'Cap.NET_ACCESS' would fail here.
    agent.kernel.check_policy(Cap.NET_ACCESS)
    print("❌ Security Check Failed: Network access was allowed (Unexpected)")
except Exception as e:
    print(f"✅ Security Check Passed: Operation blocked by kernel. ({e})")